# TSA Chapter 5: GARCH Forecast Convergence

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_convergence/TSA_ch5_convergence.ipynb)

Multi-step GARCH forecasts converge to unconditional volatility.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# GARCH forecast convergence
returns = btc_returns
am = arch_model(returns, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
res = am.fit(disp='off')
omega = res.params['omega']; alpha = res.params['alpha[1]']; beta = res.params['beta[1]']
uncond_var = omega / max(1 - alpha - beta, 0.01)

horizons = np.arange(1, 61)
fig, ax = plt.subplots(figsize=(8, 4))
starts = [-200, -150, -100, -50]
cols = [BLUE, RED, GREEN, ORANGE]
for idx, s in enumerate(starts):
    window = returns.iloc[:s]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=60)
    var_path = np.sqrt(fcast.variance.values[-1, :])
    label_date = window.index[-1].strftime('%Y-%m-%d')
    ax.plot(horizons, var_path, color=cols[idx], lw=1.0, label=f'Start: {label_date}')

ax.axhline(np.sqrt(uncond_var), color=GRAY, ls='--', lw=1.0, label='Unconditional $\\sigma$')
ax.set_xlabel('Forecast horizon (days)'); ax.set_ylabel('$\\hat{\\sigma}_{t+h}$')
ax.set_title('GARCH Multi-Step Forecasts Converge to Unconditional Volatility', fontweight='bold')
legend_bottom(ax, ncol=3, y=-0.22)
fig.tight_layout(); fig.subplots_adjust(bottom=0.22)
save_chart(fig, 'garch_convergence'); plt.show()